**KNN ML Supervised Algorithm for Suppplier Selection**

This notebook can be used by your Datascience team to run KNN clustering on prospective suppliers (operating on 10-12 features)

In [ ]:
import streamlit as st
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.plotting import parallel_coordinates
from sqlalchemy import create_engine

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()
engine = create_engine('sqlite://', echo=False)

In [ ]:
--Prepopulate the supplier facts data directly from SNOWFLAKE
select supplier,quality,quantity,payment_method,serviceability,reputation,flexibility,financial_stability,supplier_assets,business_health,price,delivery_time,supplier_location_convenience from AUTOSCM.PUBLIC.SUPPLIERFACTS;

In [ ]:
#Write supplierfacts data to a dataframe
df = cell5.to_pandas()

In [ ]:
#Scaling to the scale 0-10
#Normalize the values across all the features being trained
df['DELIVERY_TIME'] = df['DELIVERY_TIME'] * 2
df['FLEXIBILITY'] = df['FLEXIBILITY'] * 2
df['PAYMENT_METHOD'] = df['PAYMENT_METHOD'] * 2
df['QUALITY'] = df['QUALITY'] * 2

In [ ]:
#Drop Supplier Location convenience to avoid unintended bias to the ML feature training
df1 = df.drop(["SUPPLIER_LOCATION_CONVENIENCE"], axis =1)

In [ ]:
df2 = df1

X = df2.drop(['SUPPLIER'], axis=1)

#Normalize the features to a scale of 1 for k-value determination
# Scale the features to have mean=0 and standard deviation=1, except for quantity and price
scaler = StandardScaler()
X_scaled = X.copy()
X_scaled[X_scaled.columns.difference(['QUANTITY', 'PRICE'])] = scaler.fit_transform(X[X_scaled.columns.difference(['QUANTITY', 'PRICE'])])

# Convert quantity to a rating on a scale of 0-10 (higher values are better)
X_scaled['QUANTITY'] = (X_scaled['QUANTITY'] - X_scaled['QUANTITY'].min()) / (X_scaled['QUANTITY'].max() - X_scaled['QUANTITY'].min()) * 10

# Perform k-means clustering with 2 clusters (ideal and not ideal)
kmeans = KMeans(n_clusters=2, random_state=42)
df2['cluster'] = kmeans.fit_predict(X_scaled)

In [ ]:
#print(df2[['SUPPLIER', 'cluster']])

In [ ]:
from sklearn.preprocessing import MinMaxScaler
features = ['QUALITY', 'PAYMENT_METHOD', 'SERVICEABILITY',
            'REPUTATION', 'FLEXIBILITY', 'FINANCIAL_STABILITY',
            'SUPPLIER_ASSETS', 'BUSINESS_HEALTH', 'PRICE', 'DELIVERY_TIME']

# Define equal feature weights for all parameters (sum of weights is 1)
equal_weight = 1 / len(features)
feature_weights = {feature: equal_weight for feature in features}

# Normalize the features using Min-Max scaling
scaler = MinMaxScaler()
df_normalized = scaler.fit_transform(df[features])

# Calculate the score for each supplier
df['score'] = (df_normalized * list(feature_weights.values())).sum(axis=1)

df_sorted = df.sort_values(by='score', ascending=False)
print(df_sorted[['SUPPLIER', 'score']])

supplier_grades = df_sorted.head(3)

In [ ]:
#Plot the Top 3 Suppliers
ax1 = supplier_grades.plot.scatter(x='SUPPLIER', y='score')

**To call this notebook as a function elsewhere**


#execute notebook ML_Supplier_Selection_Algo ()